In [ ]:
import glob
import cv2


In [ ]:
image_directory = "/content/images"

In [ ]:
image_paths = glob.glob(image_directory + "/*.png")
images = []
for path in image_paths:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype("float32") / 255.0
    img = (img - 0.5) / 0.5  # Normalize to the range of -1 to 1
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    images.append(img)

In [ ]:
import cv2
import numpy as np

image_paths = glob.glob(image_directory + "/*.png")
images = []

for path in image_paths:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)


    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    img_rgb = img_rgb.astype("float32") / 127.5 - 1.0

    images.append(img_rgb)


x_train = np.array(images)


print("Image shape:", x_train.shape)


Image shape: (5, 28, 28, 3)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
from PIL import Image

In [ ]:
# Generator model
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(7 * 7 * 256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=1, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding="same", activation="tanh"))  # Update output channels to 3
    return model

# Discriminator model
def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=5, strides=2, padding="same", input_shape=(28, 28, 3)))  # Update input channels to 3
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=5, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model


In [ ]:
x_train.shape

(5, 28, 28, 1)

In [ ]:

latent_dim = 100


discriminator = build_discriminator()
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"])


generator = build_generator(latent_dim)

gan = keras.Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam())


x_train = x_train.reshape(-1, 28, 28, 3)

x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5

# Training loop
batch_size = 128
epochs = 10000
sample_interval = 100






In [ ]:
for epoch in range(epochs):

    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_images = x_train[idx]


    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)

    
    discriminator_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    discriminator_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

  
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generator_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print the progress
    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}/{epochs} | Discriminator Loss: {discriminator_loss[0]} | Discriminator Accuracy: {100 * discriminator_loss[1]}%")
        print(f"Generator Loss: {generator_loss}")

        # Generate and save a sample of generated images
        if epoch % sample_interval == 0:
            # Generate images from random noise
            noise = np.random.normal(0, 1, (16, latent_dim))
            generated_images = generator.predict(noise)

            # Rescale images to 0-1 range
            generated_images = 0.5 * generated_images + 0.5

            # Save the generated images
            for i in range(generated_images.shape[0]):
                img = generated_images[i]
                img = np.clip(img * 255, 0, 255).astype('uint8')
                img = Image.fromarray(img, 'RGB')
                img.save(f"/content/gen4/{epoch}_{i}.png")


In [ ]:
!zip -r /content/gen2.zip /content/gen2
